In [1]:
!pip install transformers torch accelerate

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `nada1711` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate

In [3]:
!huggingface-cli whoami


Nada1711


In [4]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [6]:
def preprocess_essay(essay_text):
    # Remove HTML tags
    essay_text = re.sub(r'<[^>]+>', '', essay_text)
    # Normalize whitespace
    essay_text = re.sub(r'\s+', ' ', essay_text)
    # Strip leading and trailing spaces
    essay_text = essay_text.strip()
    return essay_text


In [39]:
def score_essay(essay_text):

    prompt = (
    "You are an experienced essay grader. Your task is to evaluate essays "
    "Provide a single number between 1 and 6  as the score. Avoid assigning '1' unless the essay is completely incoherent.\n\n"
    "Here is the essay to evaluate:\n\n"
    f"Essay:\n{essay_text}\n\n"
    "Your score:"
)

#  check prompt doesn't exceed max
    max_prompt_length = 1024
    if len(prompt.split()) > max_prompt_length:
        prompt = prompt[:max_prompt_length]

    inputs = tokenizer(
        prompt,
        return_tensors="pt",    #Converts the tokens into PyTorch tensors
        truncation=True,
        max_length=max_prompt_length
    ).to(device)

    outputs = model.generate(
        inputs["input_ids"], #tokenized prompt.
        max_new_tokens=50, #Limits the generated response to 50 tokens.
        temperature=0.7, #controls randomness in generation
        top_k=50,#cosider the most 50 tokens with high prob
        top_p=0.9,#selects tokens whose cumulative probability adds up to 90%
        pad_token_id=tokenizer.eos_token_id #ensures all input sequences in a batch are the same length
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    score = extract_score_from_response(response)

    if score is None or not (1 <= score <= 6):
        score = "Invalid score"

    return score, response


In [8]:
import re

def extract_score_from_response(response):
    match = re.search(r'\b([1-6])\b', response)
    if match:
        return int(match.group(1))
    else:
        print(f"Unexpected response: {response}")
        return None


In [9]:
def test_individual_essay(essay_text, expected_score=None):
    preprocessed_essay = preprocess_essay(essay_text)
    predicted_score, response = score_essay(preprocessed_essay)
    print(f"Predicted Score: {predicted_score}")
    if expected_score is not None:
        print(f"Expected Score: {expected_score}")
        print("Match!" if predicted_score == expected_score else "Mismatch.")


In [26]:
import pandas as pd

dataset_path = '/content/train.csv'
df = pd.read_csv(dataset_path, engine='python', on_bad_lines='skip')

example1=df['full_text'][25]
example1_score=df['score'][25]
print('text:' , example1)
print('Actual Score',example1_score)
test_individual_essay(example1)

text: ''The Challenge of Exploring Venus'' is a informative text, Venus called the ''Evening Star''is one of the brightest point of light in the night sky,In the solar systen Venus is the secont from our sun; Earth's twin Venus is teh close planet to Earth, and occasionally the closest in distance too Earth, Venus, and Mars.

First, Venus is a hot planet, Venus has somre persent of carbon dioxide,Venus is so closer to the sumfor that reason Venus has the hottest surface temperature of any planet in iur solar system. The temoperature is the over 800 degrees fahrenheit, and the atmosphere presure is 90 times greater than what we experience on our own planet, people can't survive.

Secondly, The scientifict However, peering at Venus from a ship orbiting or hovering safely far above the planet can povide only limited insight on ground conditions because about However the light cannot penetrate th eatmosphere and it's from photography an dvideography ineffective.

Lastly, The NASA is workin

In [25]:

example2=df['full_text'][34]
example2_score=df['score'][34]
print('text:' , example2)
print('Actual Score',example2_score)
test_individual_essay(example2)

text: Did you know that Venus was maybe once like earth? And also maybe know that "Mechanical Computers" were used back in World War II that NASA may get ther idea for a invention to study Venus ? Sounds shocking right, well let me tell how and why these two questions may sound odd, but intresting.

NASA has been working on finding ways to study Venus. They have simplified electronicsthat are made of "silicon carbide" that have been tested in a chamber that is like Venus's surface, and has lasted for about 3 weeks in the comditons as Venus's surface. They still are working on other inventions that will allow the machines to last long enough to contribute meaningfully knowledge of Venus. NASA has also wanted a ship to orbit and hover safely far above the planet so that it can provide only limited insight on ground conditions because most forms of photography and videography of Venus. Astronomers have wanted to return to Venus but they wasnt a mission safe and scientifically productive. 

In [29]:

example=df['full_text'][40]
example_score=df['score'][40]
print('text:' , example)
print('Actual Score',example_score)
test_individual_essay(example)

text: To the senator of florida,

I believe that the electoral college should be abolished. The system is unjust and prevents we, the people to have our individual voices heard. When americans vote they're supposebly voting for the next president, when indeed they're votes only sway the opinion of the elector.

Think back to a little over a decade ago, to the 2000 presidential elections. Al gore received the popular vote, meaning that he was the one that us, the americans had wanted. Due to the fact that George W. Bush, his rival, received more electoral votes, Bush won, leaving an unfavored president running the country. According to source 2, a gallup poll was taken shortly after Gore had lost the presidency, an entire 60 percent of voters would prefer to have a direct vote, rather than go through the electoral process.

Granted, that the electoral college does give a certainty of outcome and that no region on its own can vote for a president, it does give an equal chance for everybo

In [34]:

example=df['full_text'][210]
example_score=df['score'][210]
print('text:' , example)
print('Actual Score',example_score)
test_individual_essay(example)

text: The Facial Action Coding System would be a good thing for students to use in the classroom. Outside of the classroom as well. Not only is it good for class work, but it is good for video games or video surgery.

The Facial Action Coding System could recognize if a student is getting confused and alter the course. So many students using computers for school aren't getting the full potential out of their computers. They stay confused and move on hoping they don't fall behind evryone else. Dr. Huang says, "Most human communication is nonverbal, including emotional communicaton. So computars need to understand that, too."

Once again I say that the Facial Action Coding System would be a good thing to use in the classroom. Not only would it help students, but the same technology could be used for video games to make faces more expressive. Or to use with video surgey. The possibilities are endless with all the new technology coming out.    
Actual Score 2
Predicted Score: 4


In [38]:

example=df['full_text'][600]
example_score=df['score'][600]
print('text:' , example)
print('Actual Score',example_score)
test_individual_essay(example)

text: The use of technology being able to read emotional expressions of students is not valuble. It invades student's privacy. It's a likely chance that a student emotions will not change because of what the computer detects, and also some people may not like the idea of being so closely connected with a computer/ technology.

There are many times where you have probably witnessed a friend or person's face looking too calm or sad to you, and when you asked them what's wrong they tell you nothing. Eventhough when they tell you this you know something might be wrong. After a while you start to just let it go,respect their privacy and wait til they're ready to really tell you whats's on their mind. This applies the same way from a computer to a student. Some students may not want the computer to be invading their privacy and knowing what's wrong with them, they would rather keep it to themselves.

It's also a likely chance that a students emotions will not change just because the computer

In [22]:
import pandas as pd

# Load the dataset
dataset_path = '/content/train.csv'
df = pd.read_csv(dataset_path, engine='python', on_bad_lines='skip')

df_filtered = df.iloc[2000:2100]

# Create a subset from the filtered data
df_subset = df_filtered.head(100)

print(df_subset.head())
print(f"Number of rows in the subset: {len(df_subset)}")

def calculate_accuracy(df):
    correct = 0
    total = len(df)

    for _, row in df.iterrows():
        essay_text = row["full_text"]
        true_score = row["score"]  # Assuming "score" column contains ground truth
        predicted_score, _ = score_essay(essay_text)

        if predicted_score == true_score:
            correct += 1

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy on the filtered subset
accuracy = calculate_accuracy(df_subset)
print(f"Model accuracy : {accuracy:.2f}%")


     essay_id                                          full_text  score
2000  1e96953  In Science Fiction movies you may have heard o...      3
2001  1e98d70  The use of this technology in a classroom is v...      1
2002  1ea12d7  In the following essay I'm going to talk about...      2
2003  1ea7607  This facial recognition technology would prove...      4
2004  1ea8387  Driverless cars, in my opinion, do not seem li...      4
Number of rows in the subset: 100
Model accuracy : 28.00%


In [ ]:
''''''''''''''''''''''''''

In [ ]:
# Process essays in batches for efficiency
batch_size = 16  # Adjust based on your system's resources
results = []

for i in range(0, len(df), batch_size):
    batch = df["full_text"][i:i + batch_size]
    batch_scores = [score_essay(text) for text in batch]
    results.extend(batch_scores)

In [ ]:
# Add model-generated scores to the DataFrame
df["model_scores"] = results

# Save the scored essays to a new CSV
output_path = "scored_essays.csv"
df.to_csv(output_path, index=False)

print("Scoring complete. Results saved to:", output_path)

In [ ]:
df["model_scores"] = df["full_text"].apply(score_essay)

# Save results to a new CSV file
output_path = "scored_essays.csv"
df.to_csv(output_path, index=False)

In [ ]:
import re

def parse_scores(output):
    match = re.search(r"Grammar: (\d+), Coherence: (\d+), Content: (\d+)", output)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    return None, None, None

# Parse the model's scores into separate columns
df[["grammar_score", "coherence_score", "content_score"]] = df["model_scores"].apply(
    lambda x: pd.Series(parse_scores(x))
)

In [ ]:
human_scores = df[["human_grammar", "human_coherence", "human_content"]]
model_scores = df[["grammar_score", "coherence_score", "content_score"]]

correlations = human_scores.corrwith(model_scores)
print("Correlations between human and model scores:")
print(correlations)

In [ ]:
'''''''''''''''''

In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


In [ ]:
scoring_criteria = {
    "Position and Thesis Clarity": {
        "description": "Evaluates how clearly the essay presents its thesis.",
        "scale": 10
    },
    "Evidence and Examples": {
        "description": "Measures the quality of supporting evidence.",
        "scale": 10
    },
    "Language and Grammar": {
        "description": "Assesses the correctness and style of the language used.",
        "scale": 10
    }
}


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from transformers import DistilBertTokenizer, DistilBertModel

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
import pandas as pd

file_path = '/content/train.csv'

try:
    train_data = pd.read_csv(file_path)
except pd.errors.ParserError as e:
    print("ParserError:", e)


In [ ]:
train_data = pd.read_csv(file_path, engine='python', on_bad_lines='skip')


In [ ]:
import pandas as pd

# Path to the dataset file
file_path = '/content/train.csv'
file2_path = '/content/test.csv'

# Read the CSV file into a DataFrame
train_data = pd.read_csv(file_path, engine='python', on_bad_lines='skip')
test_data = pd.read_csv(file2_path)

# Inspect the first few rows
print(train_data.head())
print(train_data.info())

In [ ]:
train_data['score'].unique()


In [ ]:
def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '<NUM>', text)
    text = text.lower()
    return text

train_data['cleaned_full_text'] = train_data['full_text'].apply(preprocess_text)
train_data['essay_word_count'] = train_data['cleaned_full_text'].apply(lambda x: len(x.split()))

print("Data after Preprocessing:")
print(train_data[['full_text', 'cleaned_full_text','essay_word_count']].head())

In [ ]:
train_data['cleaned_full_text']

In [ ]:
def evaluate_essay(essay, traits, pipeline):
    scores = {}
    for trait_name, trait_details in traits.items():
        try:
            score = evaluate_trait(trait_name, trait_details, essay, pipeline)
            scores[trait_name] = score
        except Exception as e:
            scores[trait_name] = str(e)  # Capture the error if any occurs
    return scores


In [ ]:
import re
from transformers import pipeline

# Assuming you are using a specific model pipeline here, for example:
llama_pipeline = pipeline("text-generation", model="gpt2")

# Example scoring criteria (you can update it based on your needs)
scoring_criteria = {
    "Position and Thesis Clarity": {
        "description": "Evaluates how clearly the essay presents its thesis.",
        "scale": 10
    },
    "Evidence and Examples": {
        "description": "Evaluates how well the essay supports its thesis with evidence and examples.",
        "scale": 10
    },
    "Language and Grammar": {
        "description": "Evaluates the clarity, coherence, and correctness of the language used in the essay.",
        "scale": 10
    }
}

# Example essay for evaluation
sample_essay = """
The benefits of using renewable energy sources are manifold. Solar power, wind energy, and hydropower not only provide clean alternatives to fossil fuels but also help reduce greenhouse gas emissions, decrease air pollution, and create jobs in green industries. Solar power, in particular, has made significant advancements over the past decade, with reduced costs and higher efficiency. Wind farms are now generating power in many regions, and hydropower continues to be a major source of energy in areas with access to large bodies of water. While renewable energy still faces challenges, such as intermittency and initial costs, technological advances and government policies are helping to mitigate these obstacles. The transition to renewable energy is essential for the health of our planet, and with continued innovation, we can create a sustainable future for generations to come.
"""

def evaluate_trait(trait_name, trait_details, essay, pipeline):
    prompt = f"""
    You are an expert in essay scoring. Evaluate the following essay based on the given trait:

    Trait: {trait_name}
    Description: {trait_details['description']}
    Essay:
    {essay}

    Please provide a detailed evaluation of the essay for this trait. After the evaluation, provide a score between 0 and 10, where 0 is the worst and 10 is the best.

    Evaluation:
    Score (0-10):
    """

    result = pipeline(
        prompt,
        max_length=512,
        max_new_tokens=100,
        truncation=True
    )

    output_text = result[0]['generated_text'].strip()
    print("Model Output:\n", output_text)  # Log the full output

    # Look for a numeric score at the end of the output text
    match = re.search(r'\b\d+(\.\d+)?\b$', output_text)  # Search for numbers at the end

    if match:
        return float(match.group(0))  # Return the first numeric value found
    else:
        raise ValueError(f"Could not extract score from output: {output_text}")

def evaluate_essay(essay, traits, pipeline):
    scores = {}
    for trait_name, trait_details in traits.items():
        try:
            score = evaluate_trait(trait_name, trait_details, essay, pipeline)
            scores[trait_name] = score
        except Exception as e:
            scores[trait_name] = str(e)  # Capture the error if any occurs
    return scores

# Evaluate the essay
essay_scores = evaluate_essay(sample_essay, scoring_criteria, llama_pipeline)

# Print the evaluation scores
print("\nEvaluation Scores:")
for trait, score in essay_scores.items():
    print(f"{trait}: {score}")


In [ ]:
# Example of a high-quality essay for testing
high_quality_essay = """
The benefits of using renewable energy sources are manifold. Solar power, wind energy, and hydropower not only provide clean alternatives to fossil fuels but also help reduce greenhouse gas emissions, decrease air pollution, and create jobs in green industries. Solar power, in particular, has made significant advancements over the past decade, with reduced costs and higher efficiency. Wind farms are now generating power in many regions, and hydropower continues to be a major source of energy in areas with access to large bodies of water. While renewable energy still faces challenges, such as intermittency and initial costs, technological advances and government policies are helping to mitigate these obstacles. The transition to renewable energy is essential for the health of our planet, and with continued innovation, we can create a sustainable future for generations to come.
"""

# Evaluate the high-quality essay using the refined function
essay_scores_high_quality = evaluate_essay(high_quality_essay, scoring_criteria, llama_pipeline)

# Print the evaluation scores for the high-quality essay
print("\nEvaluation Scores for High-Quality Essay:")
for trait, score in essay_scores_high_quality.items():
    print(f"{trait}: {score}")


In [ ]:
# Ensure the dataset is loaded and preprocessed
# Example: Extract the first essay from the dataset for evaluation
if "clean_text" in train_data.columns:
    sample_essay = train_data["cleaned_full_text"].iloc[5]  # Use preprocessed text if available
else:
    sample_essay = preprocess_text(train_data["cleaned_full_text"].iloc[5])  # Preprocess raw text

# Display the selected essay
print("Sample Essay for Evaluation:")
print(sample_essay)

# Evaluate the sample essay using the scoring criteria
essay_scores = evaluate_essay(sample_essay, scoring_criteria, llama_pipeline)

# Print the evaluation scores
print("\nEvaluation Scores:")
for trait, score in essay_scores.items():
    print(f"{trait}: {score}")


In [ ]:
''''''''''''''''''''''''''''''''

In [ ]:
import numpy as np

def aggregate_scores(trait_scores):
    """
    Aggregate trait scores and scale them.

    Parameters:
        trait_scores (list of int): Scores for individual traits.

    Returns:
        float: Final scaled score.
    """
    avg_score = np.mean(trait_scores)
    vmin, vmax = np.percentile(trait_scores, [25, 75])
    vmin = max(vmin - 1.5 * (vmax - vmin), 0)
    vmax = min(vmax + 1.5 * (vmax - vmin), 10)
    clipped_score = np.clip(avg_score, vmin, vmax)
    scaled_score = (clipped_score - vmin) / (vmax - vmin) * (10 - 0) + 0  # Scale to 0-10
    return scaled_score

In [ ]:
def evaluate_dataset(dataset, traits, pipeline):
    for trait in traits.keys():
        dataset[f"{trait}_score"] = 0
    dataset['final_score'] = 0

    for index, row in dataset.iterrows():
        essay = row['cleaned_full_text']
        trait_scores = {trait: evaluate_trait(trait, details, essay, pipeline) for trait, details in traits.items()}
        dataset.at[index, 'final_score'] = aggregate_scores(list(trait_scores.values()))
    return dataset

In [ ]:
def evaluate_trait(trait_name, trait_details, essay, pipeline):
    prompt = f"""
    Trait: {trait_name}
    Description: {trait_details['description']}
    Essay: {essay}
    Scoring Criteria:
    {', '.join(f'{key}: {value}' for key, value in trait_details['criteria'].items())}

    Based on the scoring criteria, evaluate the trait '{trait_name}' for the given essay and assign a score (0-10).
    """
    response = pipeline(
        prompt,
        max_new_tokens=100,  # Generate up to 100 tokens
        do_sample=True,
        top_k=10,
        truncation=True,  # Ensure truncation
    )
    score = int(next((word for word in response[0]["generated_text"].split() if word.isdigit()), -1))
    return score

# Preprocess Text
train_data['cleaned_full_text'] = train_data['full_text'].apply(preprocess_text)

# Evaluate Dataset
results = evaluate_dataset(train_data, scoring_criteria, llama_pipeline)
results.to_csv("scored_essays.csv", index=False)

In [ ]:
def evaluate_essay(essay, traits, pipeline):
    """
    Evaluate an essay using Multi-Trait Specialization.

    Parameters:
        essay (str): Essay text.
        traits (dict): Dictionary of traits and their scoring criteria.
        pipeline: Hugging Face pipeline for text generation.

    Returns:
        dict: Individual trait scores and final score.
    """
    trait_scores = {}
    for trait, details in traits.items():
        score = evaluate_trait(trait, details, essay, pipeline)
        trait_scores[trait] = score

    final_score = aggregate_scores(list(trait_scores.values()))
    return {"trait_scores": trait_scores, "final_score": final_score}

# Example essay
example_essay = "Computers have revolutionized many aspects of life. They impact how we work, communicate, and think."

# Evaluate essay
results = evaluate_essay(example_essay, scoring_criteria, llama_pipeline)
print("Evaluation Results:", results)